In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = "https://mate.academy/"
response = requests.get(url)
response.raise_for_status()

In [3]:
page_content = response.text

In [4]:
soup = BeautifulSoup(page_content, 'html.parser')
course_data = []
    
courses = soup.select("ul.DropdownProfessionsList_list__8OXQk a.show-for-large")
for course in courses:
    title = course.get("title", "N/A").strip()
    href = course.get("href", "#").strip()
    if title and href:
        course_data.append({"title": title, "href": url+href})
    

In [5]:
course_data

[{'title': 'UI/UX designer', 'href': 'https://mate.academy//courses/ui-ux'},
 {'title': 'Data analyst',
  'href': 'https://mate.academy//courses/data-analytics'},
 {'title': 'QA engineer', 'href': 'https://mate.academy//courses/qa'},
 {'title': 'Python developer', 'href': 'https://mate.academy//courses/python'},
 {'title': 'Digital marketer',
  'href': 'https://mate.academy//courses/digital-marketing'},
 {'title': 'Front-end developer',
  'href': 'https://mate.academy//courses/frontend'},
 {'title': 'Full-stack developer',
  'href': 'https://mate.academy//courses/fullstack'},
 {'title': 'DevOps engineer', 'href': 'https://mate.academy//courses/devops'},
 {'title': 'Java developer', 'href': 'https://mate.academy//courses/java'},
 {'title': 'Recruiter', 'href': 'https://mate.academy//courses/recruitment'},
 {'title': 'Automation QA engineer',
  'href': 'https://mate.academy//courses/qa-automation'}]

In [40]:
for course in course_data:
    response = requests.get(course['href'])
    page_content = response.text
    soup = BeautifulSoup(page_content, 'html.parser')
    
    description_element = soup.select_one("p.typography_textMain__oRJ69.SalarySection_aboutProfession__C6ftM")
    description = description_element.text.strip() if description_element else "Description not found"

    # Retrieving type
    type_texts = [p.text.strip() for p in soup.find_all("p")]
    if "Будні з 9 до 18" in type_texts and "У вільний час" in type_texts:
        course_type = "Full-time, Flex"
    elif "Будні з 9 до 18" in type_texts:
        course_type = "Full-time"
    elif "У вільний час" in type_texts:
        course_type = "Flex"
    else:
        course_type = "Unknown"
        
    course["description"] = description
    course["type"] = course_type

In [41]:
course_data

[{'title': 'UI/UX designer',
  'href': 'https://mate.academy//courses/ui-ux',
  'description': 'Щоразу, коли ти користуєшся сайтом або застосунком, ти бачиш їхню роботу. UI/UX дизайнери створюють дизайн інтерфейсів та роблять їх інтуїтивно зрозумілими. Вони також адаптують дизайни для різних пристроїв і проводять дослідження користувачів, щоб удосконалювати свої рішення. Вражає, чи не так?',
  'type': 'Full-time, Flex'},
 {'title': 'Data analyst',
  'href': 'https://mate.academy//courses/data-analytics',
  'description': 'Щоразу, коли ти користуєшся сайтом або застосунком, Data analyst збирає і аналізує дані для покращення твого досвіду. Вони створюють звіти, які допомагають бізнесам приймати обґрунтовані рішення. Вони також користуються різними інструментами та техніками для аналізу тенденцій. Вражає, чи не так?',
  'type': 'Full-time, Flex'},
 {'title': 'QA engineer',
  'href': 'https://mate.academy//courses/qa',
  'description': 'Щоразу, коли ти користуєшся сайтом або застосунком, т

In [47]:
df = pd.DataFrame(course_data)
df

,title,href,description,type
0,UI/UX designer,https://mate.academy//courses/ui-ux,"Щоразу, коли ти користуєшся сайтом або застосу...","Full-time, Flex"
1,Data analyst,https://mate.academy//courses/data-analytics,"Щоразу, коли ти користуєшся сайтом або застосу...","Full-time, Flex"
2,QA engineer,https://mate.academy//courses/qa,"Щоразу, коли ти користуєшся сайтом або застосу...","Full-time, Flex"
3,Python developer,https://mate.academy//courses/python,"Щоразу, коли ти користуєшся сайтом або застосу...","Full-time, Flex"
4,Digital marketer,https://mate.academy//courses/digital-marketing,"Щоразу, коли ти бачиш онлайн-рекламу або отрим...","Full-time, Flex"
5,Front-end developer,https://mate.academy//courses/frontend,"Щоразу, коли ти користуєшся сайтом або застосу...","Full-time, Flex"
6,Full-stack developer,https://mate.academy//courses/fullstack,"Щоразу, коли ти користуєшся сайтом або застосу...","Full-time, Flex"
7,DevOps engineer,https://mate.academy//courses/devops,"Щоразу, коли ти користуєшся сайтом або застосу...","Full-time, Flex"
8,Java developer,https://mate.academy//courses/java,"Щоразу, коли ти користуєшся сайтом або застосу...","Full-time, Flex"
9,Recruiter,https://mate.academy//courses/recruitment,Коли ти знаходиш нову роботу чи наймаєш співро...,"Full-time, Flex"


In [48]:
def split_type(row):
    types = row['type'].split(', ')
    return pd.DataFrame({
        'title': [row['title']] * len(types),
        'href': [row['href']] * len(types),
        'description': [row['description']] * len(types),
        'type': types
    })

In [49]:
df_expanded = pd.concat(df.apply(split_type, axis=1).tolist(), ignore_index=True)
df_expanded

,title,href,description,type
0,UI/UX designer,https://mate.academy//courses/ui-ux,"Щоразу, коли ти користуєшся сайтом або застосу...",Full-time
1,UI/UX designer,https://mate.academy//courses/ui-ux,"Щоразу, коли ти користуєшся сайтом або застосу...",Flex
2,Data analyst,https://mate.academy//courses/data-analytics,"Щоразу, коли ти користуєшся сайтом або застосу...",Full-time
3,Data analyst,https://mate.academy//courses/data-analytics,"Щоразу, коли ти користуєшся сайтом або застосу...",Flex
4,QA engineer,https://mate.academy//courses/qa,"Щоразу, коли ти користуєшся сайтом або застосу...",Full-time
5,QA engineer,https://mate.academy//courses/qa,"Щоразу, коли ти користуєшся сайтом або застосу...",Flex
6,Python developer,https://mate.academy//courses/python,"Щоразу, коли ти користуєшся сайтом або застосу...",Full-time
7,Python developer,https://mate.academy//courses/python,"Щоразу, коли ти користуєшся сайтом або застосу...",Flex
8,Digital marketer,https://mate.academy//courses/digital-marketing,"Щоразу, коли ти бачиш онлайн-рекламу або отрим...",Full-time
9,Digital marketer,https://mate.academy//courses/digital-marketing,"Щоразу, коли ти бачиш онлайн-рекламу або отрим...",Flex


In [50]:
# Drop href
df_expanded = df_expanded.drop('href', axis = 1)
df_expanded

,title,description,type
0,UI/UX designer,"Щоразу, коли ти користуєшся сайтом або застосу...",Full-time
1,UI/UX designer,"Щоразу, коли ти користуєшся сайтом або застосу...",Flex
2,Data analyst,"Щоразу, коли ти користуєшся сайтом або застосу...",Full-time
3,Data analyst,"Щоразу, коли ти користуєшся сайтом або застосу...",Flex
4,QA engineer,"Щоразу, коли ти користуєшся сайтом або застосу...",Full-time
5,QA engineer,"Щоразу, коли ти користуєшся сайтом або застосу...",Flex
6,Python developer,"Щоразу, коли ти користуєшся сайтом або застосу...",Full-time
7,Python developer,"Щоразу, коли ти користуєшся сайтом або застосу...",Flex
8,Digital marketer,"Щоразу, коли ти бачиш онлайн-рекламу або отрим...",Full-time
9,Digital marketer,"Щоразу, коли ти бачиш онлайн-рекламу або отрим...",Flex


In [51]:
df_expanded.to_csv('WebScrappingMateSoup.csv', index=False)